In [1]:
import pandas

In [2]:
import pandas as pd
words = pd.read_excel('Words from GRE Vocabulary Assessment Videos.xlsx')

In [3]:
set1 = words['Group 1'][1:31]

In [4]:
set1

1           abound
2        amorphous
3          austere
4            belie
5       capricious
6         cerebral
7        congenial
8      conspicuous
9          cursory
10        daunting
11           deify
12        didactic
13     disseminate
14        feasible
15           flout
16     homogeneous
17         humdrum
18         insipid
19      loquacious
20    misanthropic
21        misnomer
22       negligent
23      obsequious
24         placate
25      proclivity
26         puerile
27        quixotic
28     spendthrift
29        taciturn
30            wary
Name: Group 1, dtype: object

In [5]:
import requests

In [6]:
url = 'https://mnemonicdictionary.com/?word=obsequious'

In [8]:
page = requests.get(url)
page.status_code

200

In [12]:
from bs4 import BeautifulSoup

In [13]:
soup = BeautifulSoup(page.text, 'html.parser')

In [18]:
soup.find_all('p')

[<p>
           obsequious - Dictionary definition and meaning for word 
           <strong>obsequious</strong></p>,
 <p>
             or obse(ssed)-Qui( yes in frech)-ous=always obsessed with saying yes to it all
             </p>,
 <p>
             ob + SEQUI + ous .. sequi means sequence where one thing "follows" the other.. and servants follow what their masters say.. 
             </p>,
 <p class="md-attribution">Powered by <a href="https://mnemonicdictionary.com">Mnemonic Dictionary</a>
 </p>,
 <p>
             seq = Suck, uio = Your, us = Ass. He sucks your ass, therefore he's obsequious.
             </p>,
 <p>
             Think "seek" because when you seek something you show attentiveness to find it. 
             </p>,
 <p>
             sounds like AP+SEEK+QUOUS.......when you seek help,you are attentive and your manner is servile
             </p>,
 <p>
             obsequious - obey sequence of work in office(obedience, dutiful).
             </p>,
 <p>
             Sounds

In [28]:
soup.find_all('p')[2].get_text()

'\n            ob + SEQUI + ous .. sequi means sequence where one thing "follows" the other.. and servants follow what their masters say.. \n            '

In [17]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="MnemonicDictionary.com - Meaning of obsequious and a memory aid (called Mnemonic) to retain that meaning for long time in our memory." name="description"/>
  <meta content="obsequious, meaning, definition, word power made easy, mnemonic, mnemonics, dictionary, gre words, gre dictionary, vocabulary, word list" name="keywords"/>
  <meta content="OpenCubicles" name="author">
   <title>
    obsequious meaning - definition of obsequious by Mnemonic Dictionary
   </title>
   <link href="https://fonts.googleapis.com/css?family=Open+Sans:400,300,600" rel="stylesheet" type="text/css"/>
   <link href="/assets/css/toolkit.css" rel="stylesheet"/>
   <link href="/assets/css/application.css" rel="stylesheet"/>
   <link crossorigin="anonymous" href="https://use.fontawesome.com/releases/v5.0.